In [1]:
#Import Packages
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Import Data
titanic_train = pd.read_csv('/Users/dilynzertuche/Desktop/VisualStudio/Kaggle Competitions/Titanic/titanic/train.csv')
titanic_test = pd.read_csv('/Users/dilynzertuche/Desktop/VisualStudio/Kaggle Competitions/Titanic/titanic/test.csv')


In [3]:
#Peak At Data
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#Look At Shape of Data Set
print('The shape of our data set is: ', titanic_train.shape)

#Descriptive Statistics for Features
titanic_train.describe()

The shape of our data set is:  (891, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
#Drop Name, Cabin, Ticket from Data Set
train_set = titanic_train.drop(['Name', 'Cabin', 'Ticket'], axis = 1)

#Create Dummy Variables
train_set = pd.get_dummies(train_set, columns = ['Sex', 'Embarked'])

#Check for Missing Observations
train_set.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
Sex_female       0
Sex_male         0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
dtype: int64

In [6]:
#Impute Missing Age Values
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

#Split Data Into Set With Age (Train) and Without Age (Test)
age_train = train_set[train_set['Age'].isnull() == False]
age_test = train_set[train_set['Age'].isnull() == True]

#Set Age As Variable of Interest
y_age = age_train['Age']

#Drop Age from Training Set
age_train.drop('Age', axis = 1, inplace = True)

#Fit the Training Model
lr.fit(age_train, y_age)

#Drop Age from Test Set
age_test.drop('Age', axis = 1, inplace = True)

#Make Predictions
age_predict = lr.predict(age_test)

#Insert Predictions Into Test Set
age_test['Age'] = age_predict

/var/folders/4v/s8kknsdd5q1b0n0lypwmbslr0000gn/T/ipykernel_18974/3934529138.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_train.drop('Age', axis = 1, inplace = True)
/var/folders/4v/s8kknsdd5q1b0n0lypwmbslr0000gn/T/ipykernel_18974/3934529138.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_test.drop('Age', axis = 1, inplace = True)
/var/folders/4v/s8kknsdd5q1b0n0lypwmbslr0000gn/T/ipykernel_18974/3934529138.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [7]:
#Drop Missing Age Observations from train_set
train_set = train_set[train_set['Age'].isnull() == False]

In [8]:
#Combine train_set with Predicted Age Values (age_test['Age'])
train_set = pd.concat([train_set, age_test], axis = 0, join = 'outer')

In [9]:
#Set Variables
X = train_set.drop('Survived', axis = 1)
x = X.values
y = train_set['Survived']
y = y.values

In [10]:
#Split Model Into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 12345)

In [11]:
#Hypertune Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(50,500),
    'max_depth': randint(1,20)
}

rf = RandomForestClassifier()

rand_search = RandomizedSearchCV(rf,
                                param_distributions = param_dist,
                                n_iter = 5,
                                cv = 5,
                                random_state = 12345)

rand_search.fit(X_train, y_train)

best_rf = rand_search.best_estimator_

print('Best Parameters: ', rand_search.best_params_)

Best Parameters:  {'max_depth': 10, 'n_estimators': 432}


In [12]:
#Fit the Model
best_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=432)

In [13]:
#Make Predictions
survival_predictions = best_rf.predict(X_test)

In [14]:
#Evaluate Model
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score

rf_training_acc = best_rf.score(X_train, y_train)
rf_testing_acc = best_rf.score(X_test, y_test)

print("accuracy on training set: %f" % best_rf.score(X_train, y_train))
print("accuracy on training set: %f" % best_rf.score(X_test, y_test))

accuracy on training set: 0.967897
accuracy on training set: 0.843284


In [15]:
#Modify titanic_test set
#Drop Name, Cabin, Ticket from Data Set
test_set = titanic_test.drop(['Name', 'Cabin', 'Ticket'], axis = 1)

#Create Dummy Variables
test_set = pd.get_dummies(test_set, columns = ['Sex', 'Embarked'])

#Check for Missing Observations
test_set.isnull().sum()


PassengerId     0
Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
Sex_female      0
Sex_male        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [16]:
#Fill In Missing Fare Value
test_set['Fare'] = test_set['Fare'].fillna(test_set['Fare'].mean())

#Fill In Missing Age Values
test_set['Age'] = test_set['Age'].fillna(test_set['Age'].mean())

#Check for Missing Observations Again
test_set.isnull().sum()


PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [17]:
#Create Predictions on test_set
test_predictions_raw = best_rf.predict(test_set)

test_pred_df = pd.DataFrame(test_predictions_raw)
passengerID = test_set['PassengerId']

test_pred = pd.concat([passengerID, test_pred_df], axis = 1, join = 'outer')

test_pred = test_pred.rename(columns={0: 'Survived'})

test_pred.to_csv('TitanicTestPredictions.csv', index = False)

/Users/dilynzertuche/anaconda3/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
